Here we do some more EDA on the UNdata that we have.

In [1]:
import pandas as pd
import pickle
DATA_FILENAME = './datasets/data.pickle'

In [2]:
with open(DATA_FILENAME, 'rb') as handle:
    data = pickle.load(handle)

In [3]:
for column in data.keys():
    print(column)

education index
hdi
gdp
life expectancy
greenhouse gas
electricity employment
energy demand
energy production
energy consumption
solar electricity production
wind electricity production
hydro electricity production
wave electricity production
nuclear electricity production
brown coal energy production
brown coal briquettes energy production
charcoal energy production
coal tar energy production
coking coal energy production
crude oil energy production
fuel oil energy production
gas diesel oils energy production
other oils energy production


#### Data about combustible production

In [4]:
combustible_sources = [
    'brown coal energy production',
    'brown coal briquettes energy production',
    'charcoal energy production',
    'coal tar energy production',
    'coking coal energy production',
    'crude oil energy production',
    'fuel oil energy production',
    'gas diesel oils energy production',
    'other oils energy production'
]

combustible_df = pd.DataFrame(columns=['country', 'year']).set_index(['country', 'year'])
combustible_df = combustible_df.join([ data[combustible_source][combustible_source] for combustible_source in combustible_sources ], how='outer').sum(axis=1)

#### Data about renewable production

In [5]:
renewable_sources = [
    'solar electricity production',
    'wind electricity production',
    'hydro electricity production',
    'wave electricity production',
]

renewable_df = pd.DataFrame(columns=['country', 'year']).set_index(['country', 'year'])
renewable_df = renewable_df.join([ data[renewable_source][renewable_source] for renewable_source in renewable_sources ], how='outer').sum(axis=1)

#### Final data

In [6]:
# How many years to be considered when talking about "last years"
NUMBER_LAST_YEARS = 5

df = data['energy consumption'].join([data['energy production'], data['nuclear electricity production']], how='outer')
df['combustible production'] = combustible_df
df['renewable production'] = renewable_df
df = df.reset_index()
countries = df['country'].unique()

# key: country
# value: dict where key={'energy consumption', 'combustible production', 'renewable production'}
#               and value is float with respective value
last_years = {}

for country in countries:
    
    # Take data from last NUMBER_LAST_YEARS years
    last_years_data = df[df['country'] == country].tail(NUMBER_LAST_YEARS).sum()
    
    last_years[country] = {
        'energy consumption': last_years_data['energy consumption'],
        'combustible production': last_years_data['combustible production'],
        'renewable production': last_years_data['renewable production'],
        'nuclear production': last_years_data['nuclear electricity production'],
        'total production': last_years_data['energy production'],
    }

In [7]:
number_countries = len(last_years)

# Countries where the combustible energy production is greater or equal (ge) than the total energy consumption
combustible_production_ge_90_consumption = []

# Countries where the renewable energy production is greater or equal (ge) than the total energy consumption
renewable_production_ge_90_consumption = []

# Countries where the nuclear energy production is greater or equal (ge) than the total energy consumption
nuclear_production_ge_90_consumption = []

# Countries where the renewable energy production is greater or equal (ge) than the 80% of total energy production
renewable_production_ge_90_total = []

for country, country_data in last_years.items():
    if country_data['combustible production'] >= country_data['energy consumption'] * .9:
        combustible_production_ge_90_consumption += [country]
        
    if country_data['renewable production'] >= country_data['energy consumption'] * .9:
        renewable_production_ge_90_consumption += [country]
        
    if country_data['nuclear production'] >= country_data['energy consumption'] * .9:
        nuclear_production_ge_90_consumption += [country]
        
    if country_data['renewable production'] >= country_data['total production'] * .9:
        renewable_production_ge_90_total += [country]
        
print(f"Among {number_countries} countries, in the last {NUMBER_LAST_YEARS} years available in the dataset:")

print(f" - {len(combustible_production_ge_90_consumption)} have a combustible production >= 90% total consumption")
for country in combustible_production_ge_90_consumption:
    print(f"\t\t{country}")

print(f" - {len(renewable_production_ge_90_consumption)} have a renewable production >= 90% total consumption")
for country in renewable_production_ge_90_consumption:
    print(f"\t\t{country}")

print(f" - {len(nuclear_production_ge_90_consumption)} have a nuclear production >= 90% total consumption")
for country in nuclear_production_ge_90_consumption:
    print(f"\t\t{country}")

print(f" - {len(renewable_production_ge_90_total)} have a renewable production >= 90% total production")
for country in renewable_production_ge_90_total:
    print(f"\t\t{country}")

Among 243 countries, in the last 5 years available in the dataset:
 - 56 have a combustible production >= 90% total consumption
		Algeria
		Angola
		Australia
		Azerbaijan
		Bosnia and Herzegovina
		Brunei Darussalam
		Bulgaria
		Burundi
		Central African Rep.
		Chad
		Colombia
		Comoros
		Congo
		Curaçao
		Czechoslovakia (former)
		Ecuador
		Equatorial Guinea
		Gabon
		German Dem. R. (former)
		Greece
		Guinea-Bissau
		Haiti
		Iran (Islamic Rep. of)
		Iraq
		Kazakhstan
		Kosovo
		Kuwait
		Lao People's Dem. Rep.
		Liberia
		Libya
		Mongolia
		Neth. Antilles (former)
		Niger
		Nigeria
		North Macedonia
		Oman
		Pacific Islands (former)
		Qatar
		Russian Federation
		Rwanda
		Saudi Arabia
		Serbia
		Serbia and Montenegro
		Sierra Leone
		Somalia
		South Sudan
		Sudan (former)
		Suriname
		Timor-Leste
		Turkmenistan
		United Arab Emirates
		Venezuela (Bolivar. Rep.)
		Yemen
		Yemen Arab Rep. (former)
		Yemen, Dem. (former)
		Yugoslavia, SFR (former)
 - 25 have a renewable production >= 90